In [13]:
import os
import cv2
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import InceptionResnetV1, MTCNN
from scipy.spatial.distance import cosine
import sqlite3
from datetime import datetime
import time
from collections import defaultdict

In [14]:
conn = sqlite3.connect('attendance.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS attendance (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    timestamp TEXT
)
''')
conn.commit()

In [15]:
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

In [16]:
known_faces = {} 
dataset_path = 'dataset/' 

for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if os.path.isdir(person_path):
        embeddings = []  
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            if image_path.endswith('.jpg'):
                img = Image.open(image_path)
                img_rgb = img.convert('RGB')
                faces = mtcnn(img_rgb)
                if faces is not None:
                    face_tensor = faces[0]
                    face_tensor = face_tensor.unsqueeze(0)  
                    embedding = model(face_tensor).detach().numpy()
                    embeddings.append(embedding)
        if embeddings:
            known_faces[person_name] = np.mean(embeddings, axis=0)

print("Veritabanı oluşturuldu:", known_faces.keys())

Veritabanı oluşturuldu: dict_keys(['abdullatif_akkaya', 'ahmet_furkan_aylac', 'ahmet_kadir_cicek', 'alperen_gozum', 'alper_isleyen', 'emre_salaman', 'enes_berk_demirci', 'gokhan_guney', 'oguzhan_cinar', 'serhat_derya', 'seyma_hacifettahoglu'])


In [ ]:
def show_attendance():
    cursor.execute('SELECT name, timestamp FROM attendance ORDER BY timestamp DESC')
    records = cursor.fetchall()
    print("\nYoklama Listesi:")
    print("Name\t\t\tTimestamp")
    print("-" * 40)
    for name, timestamp in records:
        print(f"{name}\t\t{timestamp}")

def compare_faces(embedding, known_faces):
    min_distance = float('inf')
    name = None
    for known_name, known_embedding in known_faces.items():
        distance = cosine(embedding.flatten(), known_embedding.flatten()) 
        if distance < min_distance:
            min_distance = distance
            name = known_name
    if min_distance < 0.3: 
        return name
    else:
        return "Tanımlanamayan Kişi"

recorded_names = set()

def add_attendance(name):
    if name in recorded_names:
        return
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute('INSERT INTO attendance (name, timestamp) VALUES (?, ?)', (name, current_time))
    conn.commit()
    recorded_names.add(name)
    print(f"{name} için yoklama kaydedildi: {current_time}")

verification_counts = defaultdict(int)
verification_threshold = 5

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Kamera açılamadı!")
    exit()

try:
    prev_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Kare okunamadı!")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_frame = Image.fromarray(frame_rgb)

        boxes, _ = mtcnn.detect(pil_frame)
        if boxes is not None:
            draw = ImageDraw.Draw(pil_frame)
            for box in boxes:
                draw.rectangle(box.tolist(), outline=(255, 0, 0), width=3)
                face = pil_frame.crop((box[0], box[1], box[2], box[3]))
                faces = mtcnn(face) 
                if faces is not None:
                    face_embedding = model(faces[0].unsqueeze(0)).detach().numpy()  
                    name = compare_faces(face_embedding, known_faces)

                    font = ImageFont.truetype("arial.ttf", 30)
                    draw.text((box[0], box[1] - 30), name, fill=(255, 0, 0), font=font)

                    if name != "Tanımlanamayan Kişi":
                        verification_counts[name] += 1
                        if verification_counts[name] >= verification_threshold:
                            add_attendance(name)
                            verification_counts[name] = 0
                    else:
                        verification_counts = defaultdict(int)

        frame_with_boxes = cv2.cvtColor(np.array(pil_frame), cv2.COLOR_RGB2BGR)

        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time

        cv2.putText(frame_with_boxes, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Face Recognition', frame_with_boxes)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if cv2.waitKey(1) & 0xFF == ord('r'):
            show_attendance()
finally:
    cap.release()
    cv2.destroyAllWindows()

alper_isleyen için yoklama kaydedildi: 2025-05-19 13:20:54


KeyboardInterrupt: 